# Step 1: fine-tune LLM using top result from (fixed) ranker

In [2]:
def launch():
    import os
    import StepOne
    import torch

    os.environ['MODEL_TYPE'] = 'base'
    augment = int(os.environ.get('AUGMENT', '1'))
    os.environ['BATCH_SIZE'] = '16' # '32'
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepOne.step_one,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()

******** augment = 8 max_iteration = 5 model_type = base *********
n                 iter (since)         4 loss (since)       4 rouge1 (since) 4 rouge1 (dev) (since)      dt
1                0.000 (0.000)          3.119 (3.119)          0.152 (0.152)          0.000 (0.000)  2.06 m
2                0.000 (0.000)          3.217 (3.315)          0.136 (0.120)          0.000 (0.000)  4.64 m
4                0.000 (0.000)          3.186 (3.154)          0.145 (0.153)          0.000 (0.000)  7.95 m
8                0.000 (0.000)          3.112 (3.038)          0.158 (0.170)          0.000 (0.000)  15.1 m
16               0.000 (0.000)          3.063 (3.014)          0.165 (0.173)          0.000 (0.000)  29.6 m
32               0.000 (0.000)          3.020 (2.977)          0.171 (0.178)          0.000 (0.000)  57.8 m
64               0.000 (0.000)          2.966 (2.913)          0.178 (0.184)          0.000 (0.000)  1.96 h
128              0.000 (0.000)          2.940 (2.914)          0.181 

# Step 2: learn ranker using (fixed pre-finetuned) task LLM

In [ ]:
def launch():
    import os
    import StepTwo
    import torch

    os.environ['MODEL_TYPE'] = 'base'
    augment = int(os.environ.get('AUGMENT', '1'))
    os.environ['BATCH_SIZE'] = '1'
    os.environ['LEARN_BATCH_SIZE'] = '1'
    os.environ['GRAD_FREE_BATCH_SIZE'] = '32'
    os.environ['STEP1_ITER'] = '0_augment8'
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepTwo.step_two,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()